# Datapackages

Kernel: `bw25`

In [ ]:
import bw_processing as bwp
import matrix_utils as mu
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import numpy as np
import seaborn as sb
import pandas as pd
from pathlib import Path

Before we dive into it, let's think about what we need to actually build a matrix. What specific data would you need? What don't you need?

## Exercise

Please think about the minimal set of information you would need to build a *sparse matrix* using [scipy.sparse.coo_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) (sparse matrices store only non-zero values). Then, create this information as Numpy arrays and actually build a sparse matrix.

Here is the matrix you should build:

$$\begin{bmatrix} 0 & 1 \\ 2 & 3 \end{bmatrix}$$

## Hint

You will need three Numpy arrays: one for the data, one for the row indices, and one for the column indices.

## Solution

In [ ]:
from scipy import sparse

data = np.array([1, 2, 3])
rows = np.array([0, 1, 1])
cols = np.array([1, 0, 1])

matrix = sparse.coo_matrix((data, (rows, cols)), (2, 2))
matrix.toarray()

## `bw_processing`

We can run into difficulties when we want to store this data. The library `bw_processing` helps us create data packages, which can store this matrix-building data on variety of file systems. You can read the [`bw_processing` README](github.com/brightway-lca/bw_processing) for more information, and can see the [PyFilesystem2 Docs](https://docs.pyfilesystem.org/en/latest/) for more on the filesystems that can be used.

Let's define this same matrix in `bw_processing`.

Matrices by definition are two-dimensional, so we know that to build matrices we will always need to specify the row and column indices of the data. We combine these two arrays into a single Numpy [structured array](https://numpy.org/doc/stable/user/basics.rec.html), which uses the labels `row` and `col`.

In [ ]:
import bw_processing as bwp
import numpy as np

indices_array = np.array([(0, 1), (1, 0), (1, 1)], dtype=bwp.INDICES_DTYPE)
indices_array

In [ ]:
indices_array['row']

In [ ]:
bwp.INDICES_DTYPE

The data array is the same as before:

In [ ]:
data_array = np.array([1, 2, 3])
data_array

This is all we need to create a data package:

In [ ]:
dp = bwp.create_datapackage()

dp.add_persistent_vector(
    matrix="some name",
    data_array=data_array,
    name="some name",
    indices_array=indices_array,
)

For such simple matrices, we can also use a shortcut:

But before this gets too abstract, let's do the same for an example:

<img src='simple-graph.png' width='400'>

In [ ]:
natural_gas = 101
carbon_fibre = 102
bike = 103
co2 = 201

In our technosphere matrix, we will have three production exchanges (each of amount 1), and two consumption exchanges. Our biosphere matrix will only have one number, the emission of $CO_{2}$ from carbon fibre production.

Our matrices should look like this:

## Technosphere matrix

Dimensions are products (rows) by activities (columns).

$$
\left[\begin{array}{ccc} 
1 & 0 & 0\\
-237 & 1 & 0\\
0 & -2.5 & 1\\
\end{array}\right]
$$

## Biosphere matrix

Dimensions are flows (rows) by activities (columns).

$$
\left[\begin{array}{ccc} 
0 & 26.6 & 0\\ 
\end{array}\right]
$$ 

## Characterization matrix

Dimensions are flows (rows) by flows (columns).

$$
\left[\begin{array}{c} 
1\\ 
\end{array}\right]
$$

# A first LCA calculation

To use datapackages, we need to know four thing for each point: the row and column ids, the data value, and the sign. We only need to store the non-zero points. Here is an example for the technosphere matrix:

In [ ]:
t_data = np.array([
    1,   # production of natural gas
    1,   # production of carbon fibre
    1,   # production of bike
    237, # input of natural gas
    2.5, # input of carbon fibre
])
t_indices = np.array([
    (101, 101), # production of natural gas
    (102, 102), # production of carbon fibre
    (103, 103), # production of bike
    (101, 102), # input of natural gas
    (102, 103), # input of carbon fibre
    ], 
    dtype=bwp.INDICES_DTYPE
)
t_flip = np.array([False, False, False, True, True]) # Numerical sign of the inputs needs to be flipped negative

And similarly for the other matrices (no need to flip signs, so we skip that part):

In [ ]:
b_data = np.array([26.6])
b_indices = np.array([
    (201, 102), # emission of CO2
    ], 
    dtype=bwp.INDICES_DTYPE
)

In [ ]:
c_data = np.array([1])
c_indices = np.array([
    (201, 201), # CF of CO2
    ], 
    dtype=bwp.INDICES_DTYPE
)

We can now create our datapackage, and add the data for all three matrices to it:

In [ ]:
dp_static = bwp.create_datapackage()

In [ ]:
dp_static.add_persistent_vector(
    matrix='technosphere_matrix',
    indices_array=t_indices,
    data_array=t_data,
    flip_array=t_flip,
)
dp_static.add_persistent_vector(
    matrix='biosphere_matrix',
    indices_array=b_indices,
    data_array=b_data,
)
dp_static.add_persistent_vector(
    matrix='characterization_matrix',
    indices_array=c_indices,
    data_array=c_data,
)

This is already enough to calculate an LCA score:

In [ ]:
lca = bc.LCA(
    demand={bike: 1},
    data_objs=[dp_static],
)
lca.lci()
lca.lcia()
lca.score

# Uncertainty

Datapackages allow for uncertainty to expressed in multiple ways; the classic way is LCA is with probability distribution functions, but it can also be given via population data or samples drawn from nonparameteric distributions.

# Using arrays for scenarios

We can use arrays for scenarios. Let's imagine two different possibilities: a lightweight bike with 1.5 kilograms of carbon fibre, and a technology platform that allows for efficient bike sharing making each bike functionally equivalent to two bikes (I know, I find this silly as well). If we treat these as separate possibilities, we have four scenarios in total.

For this to work, we will need to create **two** new arrays, one for each choice, and then tell the software to do combinatorial sampling:

In [ ]:
dp_scenarios = bwp.create_datapackage(combinatorial=True)

In [ ]:
dp_scenarios.add_persistent_array(
    matrix='technosphere_matrix',
    indices_array=np.array([(102, 103)], dtype=bwp.INDICES_DTYPE),
    data_array=np.array([(2.5, 1.5)]),
    flip_array=np.array([True]),
    name='cf scenario'
)
dp_scenarios.add_persistent_array(
    matrix='technosphere_matrix',
    indices_array=np.array([(103, 103)], dtype=bwp.INDICES_DTYPE),
    data_array=np.array([(1, 2)]),
    name='double bike'
)

In [ ]:
scenario_mapping = {
    (0, 0): "Original",
    (0, 1): "Each bike counts double",
    (1, 0): "Lightweight",
    (1, 1): "Lightweight & each bike counts double",
}

In [ ]:
lca = bc.LCA(
    demand={bike: 1},
    data_objs=[dp_static, dp_scenarios],
    use_arrays=True,
)
lca.lci()
lca.lcia()

In [ ]:
resource_group = next(grp for grp in lca.technosphere_mm.groups if grp.label == 'double bike').indexer.indexer

In [ ]:
print(lca.score, scenario_mapping[resource_group.index])

for scenario_result in lca:
    print(lca.score, scenario_mapping[resource_group.index])

# Datapackage structure

Let's look at a real-world example.

In [ ]:
bd.projects.set_current("datapackage demo")

In [ ]:
bi.useeio11()

In [ ]:
useeio = bd.Database("US EEIO 1.1")

In [ ]:
htox = bd.Method(('Impact Potential', 'HTOX'))

In [ ]:
prod = useeio.get(id=540)
prod, prod['type']

In [ ]:
lca = bc.LCA({prod.id: 1}, data_objs=[useeio.datapackage(), htox.datapackage()])
lca.lci()
lca.lcia()
lca.score

`metadata` attribute:

In [ ]:
dp = useeio.datapackage()
dp.metadata

Also have `resources` (just part of the `metadata`).

`data` attribute:

In [ ]:
dp.data

## Accessing specific resources (data and resource metadata)

In [ ]:
array, dict_ = dp.get_resource('US_EEIO_1.1_biosphere_matrix.data')

In [ ]:
array

In [ ]:
dict_

## Modifying existing data packages

Once a datapackage is loaded (or created), you can change the arrays in place. Let's reduce the inputs of out functional unit by half:

In [ ]:
tm_indices, _ = dp.get_resource('US_EEIO_1.1_technosphere_matrix.indices')
tm_data, _ = dp.get_resource('US_EEIO_1.1_technosphere_matrix.data')
tm_flip, _ = dp.get_resource('US_EEIO_1.1_technosphere_matrix.flip')

In [ ]:
act = next(exc.output for exc in prod.consumers() if exc.output['name'] == exc.input['name'])

In [ ]:
mask = (tm_indices['col'] == act.id) * tm_flip
mask.sum()

In [ ]:
tm_data[mask] *= 0.5

In [ ]:
lca2 = bc.LCA({prod.id: 1}, data_objs=[useeio.datapackage(), htox.datapackage()])
lca2.lci()
lca2.lcia()
lca.score, lca2.score

# Saving modified datapackages

In general you should save new copies instead of saving changes to existing datapackages, especially ones stored as ZipFiles (they are just a pain to deal with).

In [ ]:
ndp = bwp.create_datapackage(
    fs=bwp.generic_zipfile_filesystem(dirpath=Path.cwd(), filename="modified.zip", write=True),
)
ndp.add_persistent_vector(
    matrix="technosphere_matrix",
    indices_array=tm_indices,
    data_array=tm_data,
    flip_array=tm_flip
)
ndp.finalize_serialization()

In [ ]:
lca3 = bc.LCA(
    {prod.id: 1}, data_objs=[
        bwp.load_datapackage(fs_or_obj=bwp.generic_zipfile_filesystem(dirpath=Path.cwd(), filename="modified.zip", write=False)), 
        htox.datapackage()
    ]
)
lca3.lci()
lca3.lcia()
lca.score, lca3.score

# Challenge

Create and use a datapackage with *just* the modified data (same method, functional unit, etc.)

# More links:

Docs: https://github.com/brightway-lca/bw_processing

More simple use cases: https://github.com/brightway-lca/from-the-ground-up/blob/main/2%20-%20Building%20and%20using%20matrices%20in%20bw2calc.ipynb

Interfaces: https://github.com/brightway-lca/matrix_utils/blob/main/dev/Brightway%202.5%20demonstration.ipynb

Modify supply chains (more real than our example): https://github.com/brightway-lca/matrix_utils/blob/main/dev/Supply%20chain%20modification.ipynb